In [2]:
import pandas as pd
dataset = pd.read_csv('hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
dataset.shape

(5242, 3)

In [4]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [5]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1,"_",tweet)

1 _  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 _ we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 _ i get to see my daddy today!!   #80days #gettingfed
4 _ ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 _ i am thankful for having a paner. #thankful #positive     


In [6]:
import re
def clean_text(text):
  text = re.sub(r'[^a-zA-Z]', '', text)
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)
  text = text.lower()
  return text

In [7]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [8]:
dataset.head(10)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,userwhenafatherisdysfunctionalandissoselfishhe...
1,2,0,@user @user thanks for #lyft credit i can't us...,useruserthanksforlyftcrediticantusecausetheydo...
2,3,0,bihday your majesty,bihdayyourmajesty
3,4,0,#model i love u take with u all the time in ...,modeliloveutakewithuallthetimeinur
4,5,0,factsguide: society now #motivation,factsguidesocietynowmotivation
5,6,0,[2/2] huge fan fare and big talking before the...,hugefanfareandbigtalkingbeforetheyleavechaosan...
6,7,0,@user camping tomorrow @user @user @user @use...,usercampingtomorrowuseruseruseruseruseruseruse...
7,8,0,the next school year is the year for exams.ð...,thenextschoolyearistheyearforexamscantthinkabo...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,wewonlovethelandallincavschampionsclevelandcle...
9,10,0,@user @user welcome here ! i'm it's so #gr...,useruserwelcomehereimitssogr


In [9]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
stop = stopwords.words('english')

In [12]:
def gen_freq(text):
  word_list = []
  for tw_word in text.split():
    word_list.extend(tw_word)
  word_freq = pd.Series(word_list).value_counts()
  word_freq = word_freq.drop(stop, errors='ignore')
  return word_freq

In [13]:
def any_neg(words):
  for word in words:
    if word in ['no', 'nor', 'not'] or re.search(r"\wn't", word):
      return 1
  else:
    return 0

In [14]:
def any_rare(words, rare_100):
  for word in words:
    if word in rare_100:
      return 1
  else:
    return 0

In [15]:
def is_question(words):
  for word in words:
    if word in ['when', 'what', 'how', 'why', 'who']:
      return 1
  else:
    return 0

In [16]:
word_freq = gen_freq(dataset.clean_text.str)
rare_100 = word_freq[-100:]
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))
dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))
dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))

In [17]:
dataset.head(10)

,id,label,tweet,clean_text,word_count,any_neg,is_question,any_rare,char_count
0,1,0,@user when a father is dysfunctional and is s...,userwhenafatherisdysfunctionalandissoselfishhe...,1,0,0,0,79
1,2,0,@user @user thanks for #lyft credit i can't us...,useruserthanksforlyftcrediticantusecausetheydo...,1,0,0,0,93
2,3,0,bihday your majesty,bihdayyourmajesty,1,0,0,0,17
3,4,0,#model i love u take with u all the time in ...,modeliloveutakewithuallthetimeinur,1,0,0,0,34
4,5,0,factsguide: society now #motivation,factsguidesocietynowmotivation,1,0,0,0,30
5,6,0,[2/2] huge fan fare and big talking before the...,hugefanfareandbigtalkingbeforetheyleavechaosan...,1,0,0,0,88
6,7,0,@user camping tomorrow @user @user @user @use...,usercampingtomorrowuseruseruseruseruseruseruse...,1,0,0,0,52
7,8,0,the next school year is the year for exams.ð...,thenextschoolyearistheyearforexamscantthinkabo...,1,0,0,0,104
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,wewonlovethelandallincavschampionsclevelandcle...,1,0,0,0,61
9,10,0,@user @user welcome here ! i'm it's so #gr...,useruserwelcomehereimitssogr,1,0,0,0,28


In [18]:
from sklearn.model_selection import train_test_split
X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=0)

In [19]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [20]:
model.predict(X_test[5:10])

array([1, 1, 1, 1, 1])

In [21]:
from sklearn.metrics import accuracy_score
print("Accuracy Score: ", accuracy_score(y_test, y_pred)*100, "%")

Accuracy Score:  44.518589132507145 %


In [22]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
rf_pred = clf.predict(X_test).astype(int)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, \
accuracy_score
print(confusion_matrix(y_test,rf_pred))
print(classification_report(y_test,rf_pred))
print("Accuracy:",accuracy_score(y_test, rf_pred))

[[430 169]
 [233 217]]
              precision    recall  f1-score   support

           0       0.65      0.72      0.68       599
           1       0.56      0.48      0.52       450

    accuracy                           0.62      1049
   macro avg       0.61      0.60      0.60      1049
weighted avg       0.61      0.62      0.61      1049

Accuracy: 0.6167778836987607


In [24]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [25]:
y_pred = logreg.predict(X_test) #predicting the values

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.53      0.58       599
           1       0.50      0.63      0.56       450

    accuracy                           0.57      1049
   macro avg       0.58      0.58      0.57      1049
weighted avg       0.59      0.57      0.57      1049

